In [1]:
import sys
import pandas as pd
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import homogeneity_score,completeness_score,v_measure_score,accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import RFE

TRAIN_TEST_RATIO = 0.25

In [2]:

data = pd.read_csv('dermatology.data.txt', sep=",", header = None)

# [1-11,34] podaci pre analiza
# [12-33] podaci nakon analiza
data.columns = cols = ["erythema", "scaling", "definite borders", "itching","koebner phenomenon",
                "polygonal papules","follicular papules","oral mucosal involvement","knee and elbow involvement",
                "scalp involvement","family history","melanin incontinence","eosinophils in the infiltrate","PNL infiltrate",
                "fibrosis of the papillary dermis","exocytosis","acanthosis","hyperkeratosis","parakeratosis","clubbing of the rete ridges",
               "elongation of the rete ridges","thinning of the suprapapillary epidermis","spongiform pustule","munro microabcess","focal hypergranulosis",
               "disappearance of the granular layer","vacuolisation and damage of basal layer","spongiosis","saw-tooth appearance of retes","follicular horn plug",
               "perifollicular parakeratosis","inflammatory monoluclear inflitrate","band-like infiltrate","Age","des_class"]

# data.head()

#izbaciti redove koje sadrze '?' u koloni za godine
# print(len(data))
data = data[data.Age != '?']
print(len(data))
data = data[data.des_class != 6]
print(len(data))

#izbaciti godine za probu

# data = data.drop('Age',1)


results = data['des_class']

#normalize




cols = data.columns

min_max_scaler = MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)


data.columns = cols

attributes = data.drop('des_class',1)
results.head()
# print(attributes[attributes.columns[1]])
# attributes.head()
# print(len(results))

358
338


0    2
1    1
2    3
3    1
4    3
Name: des_class, dtype: int64

In [3]:

train_data, test_data,train_results,test_results = train_test_split(attributes, results ,test_size=TRAIN_TEST_RATIO)


print('%d - %d' % (len(train_results), len(test_results)))


253 - 85


In [4]:
#izbaciti atrbiute dobijene biopsijom
# for i in range(12,33):
#     train_data = train_data.drop(cols[i],1)
#     test_data = test_data.drop(cols[i],1)

# naci najbitinije feature
model = ExtraTreesClassifier(n_estimators=250)
model.fit(data, results)
print(model.feature_importances_)
# cols = deepcopy(train_data.columns)
# print(len(cols))
# for idx,elem in enumerate(model.feature_importances_):
    
#     if float(elem) < float(0.01):
#         print(cols[idx])
       
#         train_data = train_data.drop(cols[idx],1)
#         test_data = test_data.drop(cols[idx],1)


model = LogisticRegression()
rfe = RFE(model, 10)
fit = rfe.fit(data, results)
print(fit.n_features_)
print(fit.support_)
print(fit.ranking_)

cols = deepcopy(data.columns)

print(len(cols))

# for idx,elem in enumerate(fit.support_):
    
#     if not elem:
#         print(elem) 
#         print(idx)
#         print(cols[idx])
       
#         train_data = train_data.drop(cols[idx],1)
#         test_data = test_data.drop(cols[idx],1)




[  4.35276875e-03   1.23560306e-02   8.01200556e-03   1.64915794e-02
   3.80793902e-02   3.20799622e-02   1.75367423e-03   2.96678610e-02
   2.11807521e-02   2.00478903e-02   2.13222294e-03   3.45767069e-02
   6.74981900e-03   2.05462559e-02   5.80737102e-02   2.76374836e-02
   5.00536833e-03   3.09013139e-03   8.41258694e-03   7.34198486e-02
   5.84399018e-02   6.64831917e-02   7.10505098e-03   1.61268284e-02
   3.62320027e-02   2.25380476e-02   3.80107586e-02   5.72839284e-02
   3.83263714e-02   5.21670812e-04   2.45530168e-05   2.54844209e-03
   4.58934673e-02   2.41730266e-03   1.84382435e-01]
10
[False False False False  True  True False False False False False False
 False False  True False False False False  True  True  True False False
 False False False  True  True False False False  True False  True]
[16  5 14  2  1  1 21  8 10 12 24  9 13  3  1  6 15 23 22  1  1  1 19 17 11
  7  4  1  1 25 26 18  1 20  1]
35


In [9]:
kmeans = SVC(kernel='rbf', probability=True)
neigh = KNeighborsClassifier(n_neighbors=10)
gnb = GaussianNB()



vclf = VotingClassifier(estimators=[('kmeans', kmeans), ('neigh', neigh), ('gnb', gnb)], voting='soft',
                        weights=[1,1,1])
vclf.fit(train_data,train_results)

VotingClassifier(estimators=[('kmeans', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('neigh', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')), ('gnb', GaussianNB())],
         voting='soft', weights=[1, 1, 1])

In [10]:
predicted = vclf.predict(test_data)

print(accuracy_score(predicted,test_results))


import sklearn.metrics as sm
print(sm.confusion_matrix(test_results,predicted))



0.952941176471
[[33  0  0  0  0]
 [ 0 12  0  4  0]
 [ 0  0 15  0  0]
 [ 0  0  0 12  0]
 [ 0  0  0  0  9]]
